In [3]:
pip install requests pymongo sqlalchemy

  Using cached pymongo-4.7.2-cp38-cp38-macosx_10_9_x86_64.whl.metadata (22 kB)
  Using cached SQLAlchemy-2.0.30-cp38-cp38-macosx_10_9_x86_64.whl.metadata (9.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.5/486.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.3 MB/s eta 0:00:00
  Created wheel for greenlet: filename=greenlet-3.0.3-cp38-cp38-macosx_10_9_x86_64.whl size=218129 sha256=cd5a3f65c319f050987da5a0e39a713e3d8ede10ec9c982dbfc2370d5a9d81ea
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/48/c3/46/5e61d8224ca5247a7464bdd6fdded397df3f17812d0cbb402f
Successfully built greenlet


In [2]:
import pandas as pd
import time
from sqlalchemy import create_engine, Column, Float, String, Integer, DateTime, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
# from polygon import RESTClient
from datetime import datetime
import pymongo
from pymongo import MongoClient
import requests

In [3]:
#set up the API
api_key = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'
currency_pairs = ['AUD/USD', 'EUR/USD', 'GBP/USD']

In [4]:
def fetch_fx_rate(base, quote, api_key):
    url = f'https://api.polygon.io/v1/conversion/{base}/{quote}?amount=100&precision=2&apiKey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'converted' in data:
            return data['converted']
    return None

In [5]:
# Connect to SQLite database using SQLAlchemy
engine = create_engine('sqlite:///forex_data_v1.db')
metadata = MetaData()

forex_rates = Table('forex_rates', metadata,
                    Column('id', Integer, primary_key=True, autoincrement=True),
                    Column('currency_pair', String),
                    Column('fx_rate_timestamp', DateTime),
                    Column('fx_rate_value', Float),
                    Column('db_entry_timestamp', DateTime))

metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
# Connect to MongoDB (assuming localhost and default port)
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client['DataEngineering']
collection = db['student_v1']

In [7]:
# Define the duration for fetching data (2 hours)
end_time = time.time() + 2 * 60 * 60  # 2 hours in seconds
#end_time = time.time() + 60
while time.time() < end_time:
    for pair in currency_pairs:
        base, quote = pair.split('/')
        fx_rate_value = fetch_fx_rate(base, quote, api_key)
        fx_rate_timestamp = datetime.now()
        db_entry_timestamp = fx_rate_timestamp
        
        if fx_rate_value is not None:
            # Insert data into SQLite
            session.execute(forex_rates.insert().values(
                currency_pair=pair,
                fx_rate_timestamp=fx_rate_timestamp,
                fx_rate_value=fx_rate_value,
                db_entry_timestamp=db_entry_timestamp
            ))
            session.commit()

            # Prepare data for MongoDB
            mongo_data = {
                'currency_pair': pair,
                'fx_rate_timestamp': fx_rate_timestamp,
                'fx_rate_value': fx_rate_value,
                'db_entry_timestamp': db_entry_timestamp
            }
            
            # Insert data into the collection
            collection.insert_one(mongo_data)
            
            print(f'Real-time data stored for {pair}: {fx_rate_value} at {fx_rate_timestamp}')
    
    # Wait for a second before the next request
    time.sleep(1)

# Close MongoDB connection
mongo_client.close()

print('Data successfully stored in both SQLite and MongoDB')

Real-time data stored for AUD/USD: 66.56 at 2024-05-27 10:33:36.973208
Real-time data stored for EUR/USD: 108.54 at 2024-05-27 10:33:37.109008
Real-time data stored for GBP/USD: 127.72 at 2024-05-27 10:33:37.206254
Real-time data stored for AUD/USD: 66.56 at 2024-05-27 10:33:38.398077
Real-time data stored for EUR/USD: 108.54 at 2024-05-27 10:33:38.489830
Real-time data stored for GBP/USD: 127.72 at 2024-05-27 10:33:38.603780
Real-time data stored for AUD/USD: 66.55 at 2024-05-27 10:33:39.720085
Real-time data stored for EUR/USD: 108.54 at 2024-05-27 10:33:39.880976
Real-time data stored for GBP/USD: 127.72 at 2024-05-27 10:33:39.983201
Real-time data stored for AUD/USD: 66.56 at 2024-05-27 10:33:41.089786
Real-time data stored for EUR/USD: 108.54 at 2024-05-27 10:33:41.189795
Real-time data stored for GBP/USD: 127.72 at 2024-05-27 10:33:41.297174
Real-time data stored for AUD/USD: 66.56 at 2024-05-27 10:33:42.399409
Real-time data stored for EUR/USD: 108.54 at 2024-05-27 10:33:42.5038